# Canny Benchmark

## TODO:

* Traceability with benchmark checkpoints that are passed into canny-end-to-end

## Imports and Setup

In [39]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append("../")

import logging
from pathlib import Path
import unittest.mock as mock

from icecream import ic

from IPython.display import display
import ipywidgets as wid
from ipywidgets import Layout
from utils.ipywidgets_extended import widgets_styling, ManyBoxes

from utils.setup_notebook import init_notebook
from utils.setup_logging import setup_logging
import utils.memoize as memoize

init_notebook()
setup_logging("INFO")
memoize.set_file_store_path("canny_benchmark")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import numpy as np
import pandas as pd
import scipy as sp
import numba
from numba import cuda
import cv2

from utils.benchmarking import time_line, setup_process_for_benchmarking
from utils.plotting_tools import (
    SmartFigure,
    to_ipy_image,
    plot_kernel,
    plot_matrix,
)
import utils.dyn_module as dyn

from canny_common import load_input_images

logging.getLogger("numba.cuda.cudadrv.driver").setLevel(logging.WARNING)
setup_process_for_benchmarking()

In [38]:
reset_memoize_store_button = wid.Button(description="Reset memoize store")
reset_memoize_store_button.on_click(lambda x: memoize.reset_store())
display(reset_memoize_store_button)


Button(description='Reset memoize store', style=ButtonStyle())

## Loading Canny Implementations

In [4]:
dir_canny_impls = "./canny_impls"
canny_impls_module_names = dyn.load_modules(dir_canny_impls)

2024-10-26 16:31:39.902 INFO root Loading 2 modules started (..\utils\dyn_module.py:49)
2024-10-26 16:31:39.923 INFO root Loading rd_numba_cuda_fp32 started (..\utils\dyn_module.py:30)
2024-10-26 16:31:43.197 INFO root Loading rd_numba_cuda_fp32 took: 3.2734 s (..\utils\dyn_module.py:30)
2024-10-26 16:31:43.231 INFO root Loading rd_vec_v4_dibit started (..\utils\dyn_module.py:30)
2024-10-26 16:31:43.838 INFO root Loading rd_vec_v4_dibit took: 606.9274 ms (..\utils\dyn_module.py:30)
2024-10-26 16:31:43.839 INFO root Loading 2 modules took: 3.9361 s (..\utils\dyn_module.py:49)


## Loading Input Images

In [5]:
input_images_dir = "./image_input"
input_images = load_input_images(input_images_dir)

2024-10-26 16:31:44.215 INFO root Loading 6 images started (canny_common.py:22)
2024-10-26 16:31:44.239 INFO root Loading circle_128.png started (canny_common.py:24)
2024-10-26 16:31:44.240 INFO root Loading circle_128.png took: 1.2197 ms (canny_common.py:24)
2024-10-26 16:31:44.266 INFO root Loading circle_32.png started (canny_common.py:24)
2024-10-26 16:31:44.267 INFO root Loading circle_32.png took: 920.9000 µs (canny_common.py:24)
2024-10-26 16:31:44.307 INFO root Loading circle_512.png started (canny_common.py:24)
2024-10-26 16:31:44.317 INFO root Loading circle_512.png took: 9.8631 ms (canny_common.py:24)
2024-10-26 16:31:44.343 INFO root Loading circle_64.png started (canny_common.py:24)
2024-10-26 16:31:44.344 INFO root Loading circle_64.png took: 554.3000 µs (canny_common.py:24)
2024-10-26 16:31:44.368 INFO root Loading park.png started (canny_common.py:24)
2024-10-26 16:31:44.748 INFO root Loading park.png took: 378.9416 ms (canny_common.py:24)
2024-10-26 16:31:44.772 INFO r

## Running Benchmarks

In [49]:
fig_width = 16
image_size = 512

KEY_IMAGES_MANYBOXES = "run_image_dropdown"
images_manyboxes = ManyBoxes(
    [image.filename for image in input_images],
    memoize.get(KEY_IMAGES_MANYBOXES, default=[]),
)

KEY_SELECTED_IMPLS_MANYBOXES = "run_selected_impls"
selected_impls_manyboxes = ManyBoxes(
    canny_impls_module_names, memoize.get(KEY_SELECTED_IMPLS_MANYBOXES, default=[])
)

KEY_WARMUP_RUNS_SLIDER = "run_warmup_runs"
warmup_runs_slider = wid.IntSlider(
    value=memoize.get(KEY_WARMUP_RUNS_SLIDER, default=100),
    min=0,
    max=100,
    step=1,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
)
KEY_RUNS_SLIDER = "run_runs"
runs_slider = wid.IntSlider(
    value=memoize.get(KEY_RUNS_SLIDER, default=1000),
    min=1,
    max=10000,
    step=1,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
)

run_benchmark_button = wid.Button(description="Run benchmarks", **widgets_styling)
output = wid.Output()


def run_benchmarks(change=None):
    pass


run_benchmark_button.on_click(run_benchmarks)

display(
    wid.VBox(
        [
            wid.HTML("<h2>Image Selection</h2>"),
            images_manyboxes.get_view(),
            wid.HTML("<h2>Implementation Selection</h2>"),
            selected_impls_manyboxes.get_view(),
            wid.HBox(
                [
                    wid.VBox(
                        [
                            wid.HTML("Warmup Runs"),
                            wid.HTML("Runs"),
                        ]
                    ),
                    wid.VBox(
                        [
                            warmup_runs_slider,
                            runs_slider,
                        ]
                    ),
                ]
            ),
            run_benchmark_button,
            output,
        ]
    )
)

## Plotting Benchmarks